In [1]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.stop()

tracker.start()

[codecarbon INFO @ 17:03:20] [setup] RAM Tracking...
[codecarbon INFO @ 17:03:20] [setup] CPU Tracking...
[codecarbon WARNING @ 17:03:23] No CPU tracking mode found. Falling back on CPU constant mode. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 17:03:25] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 17:03:25] [setup] GPU Tracking...
[codecarbon INFO @ 17:03:25] No GPU found.
[codecarbon INFO @ 17:03:25] >>> Tracker's metadata:
[codecarbon INFO @ 17:03:25]   Platform system: macOS-10.16-x86_64-i386-64bit
[codecarbon INFO @ 17:03:25]   Python version: 3.9.12
[codecarbon INFO @ 17:03:25]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 17:03:25]   Available RAM : 8.000 GB
[codecarbon INFO @ 17:03:25]   CPU count: 8
[codecarbon INFO @ 17:03:25]   CPU model: Apple M1
[codecarbon INFO @ 17:03:25]   GPU count: None
[codecarbon INFO @ 17:03:25]   GPU model: None
[codecarbon INFO @ 17:03:28] Saving emissions data to

In [2]:
import pandas as pd

# Carga del CSV
df = pd.read_csv('../training.csv')  # ../ sube un nivel desde /SVM/

# Primer vistazo
print(df.shape)
df.head()


(250000, 33)


,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [3]:
import numpy as np

# Codificar la variable objetivo: 's' → 1, 'b' → 0
df['Label'] = df['Label'].map({'s': 1, 'b': 0})

# Reemplazar -999.0 por NaN
df.replace(-999.0, np.nan, inplace=True)


In [4]:
# Eliminar columnas que no usaremos
df.drop(['EventId', 'Weight'], axis=1, inplace=True)

# Separar X (features) e y (target)
X = df.drop('Label', axis=1)
y = df['Label']


In [5]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, test_size=0.2, random_state=42, stratify=y
)


In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Reducimos el conjunto de entrenamiento al 25% por tuning.

In [8]:
from sklearn.model_selection import train_test_split

# Subconjunto del 25% para tuning
X_sub, _, y_sub, _ = train_test_split(
    X_train_scaled, y_train, train_size=0.25, stratify=y_train, random_state=42
)

print(f"Subconjunto para tuning: {X_sub.shape[0]} muestras")


Subconjunto para tuning: 50000 muestras


In [9]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import time

# Lista de configuraciones a probar
# Nuevas combinaciones más variadas
svm_configuraciones = [
    {'kernel': 'rbf', 'C': 0.5, 'gamma': 'scale'},
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'}, 
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 0.1},
    {'kernel': 'poly', 'C': 1.0, 'gamma': 'scale'},
    {'kernel': 'linear', 'C': 1.0}
]


# Guardar resultados
resultados_sub = {}

for i, config in enumerate(svm_configuraciones):
    print(f"\n⏳ Entrenando SVM #{i+1}: {config}")
    inicio = time.time()
    
    modelo = SVC(**config, random_state=42)
    modelo.fit(X_sub, y_sub)
    y_pred = modelo.predict(X_test_scaled)  # ¡Seguimos evaluando en el test completo!
    
    fin = time.time()
    
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    resultados_sub[f"SVM_{i+1}_{config}"] = {
        'accuracy': acc,
        'recall_1': report['1']['recall'],
        'f1_1': report['1']['f1-score'],
        'tiempo': fin - inicio
    }

    print(f"✔️ Terminado en {fin - inicio:.2f} s - Accuracy: {acc:.4f}")



⏳ Entrenando SVM #1: {'kernel': 'rbf', 'C': 0.5, 'gamma': 'scale'}


[codecarbon INFO @ 17:03:43] Energy consumed for RAM : 0.000013 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:03:43] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:03:43] 0.000033 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:03:58] Energy consumed for RAM : 0.000025 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:03:58] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:03:58] 0.000067 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:04:13] Energy consumed for RAM : 0.000038 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:04:13] Energy consumed for all CPUs : 0.000063 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:04:13] 0.000100 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:04:28] Energy consumed for RAM : 0.000050 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:04:28] Energy consumed for all CPUs : 0.000083 kWh. Total CPU Power : 5.0 W
[codecarbon

✔️ Terminado en 183.77 s - Accuracy: 0.8242

⏳ Entrenando SVM #2: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'}


[codecarbon INFO @ 17:06:43] Energy consumed for RAM : 0.000163 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:06:43] Energy consumed for all CPUs : 0.000271 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:06:43] 0.000433 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:06:58] Energy consumed for RAM : 0.000175 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:06:58] Energy consumed for all CPUs : 0.000292 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:06:58] 0.000467 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:07:13] Energy consumed for RAM : 0.000188 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:07:13] Energy consumed for all CPUs : 0.000313 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:07:13] 0.000500 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:07:28] Energy consumed for RAM : 0.000200 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:07:28] Energy consumed for all CPUs : 0.000333 kWh. Total CPU Power : 5.0 W
[codecarbon

✔️ Terminado en 181.99 s - Accuracy: 0.8269

⏳ Entrenando SVM #3: {'kernel': 'rbf', 'C': 1.0, 'gamma': 0.1}


...

✔️ Terminado en 251.84 s - Accuracy: 0.8190

⏳ Entrenando SVM #4: {'kernel': 'poly', 'C': 1.0, 'gamma': 'scale'}


[codecarbon INFO @ 17:13:58] Energy consumed for RAM : 0.000525 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:13:58] Energy consumed for all CPUs : 0.000875 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:13:58] 0.001400 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:14:13] Energy consumed for RAM : 0.000538 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:14:13] Energy consumed for all CPUs : 0.000896 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:14:13] 0.001434 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:14:28] Energy consumed for RAM : 0.000550 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:14:28] Energy consumed for all CPUs : 0.000917 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:14:28] 0.001467 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:14:43] Energy consumed for RAM : 0.000563 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:14:43] Energy consumed for all CPUs : 0.000938 kWh. Total CPU Power : 5.0 W
[codecarbon

✔️ Terminado en 116.54 s - Accuracy: 0.7999

⏳ Entrenando SVM #5: {'kernel': 'linear', 'C': 1.0}


[codecarbon INFO @ 17:15:58] Energy consumed for RAM : 0.000625 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:15:58] Energy consumed for all CPUs : 0.001042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:15:58] 0.001667 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:16:13] Energy consumed for RAM : 0.000638 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:16:13] Energy consumed for all CPUs : 0.001063 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:16:13] 0.001701 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:16:28] Energy consumed for RAM : 0.000650 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:16:28] Energy consumed for all CPUs : 0.001084 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:16:28] 0.001734 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:16:43] Energy consumed for RAM : 0.000663 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:16:43] Energy consumed for all CPUs : 0.001105 kWh. Total CPU Power : 5.0 W
[codecarbon

✔️ Terminado en 229.17 s - Accuracy: 0.7494


In [10]:
print("\nResumen tuning con 25% de datos:")
for nombre, res in resultados_sub.items():
    print(f"\nModelo: {nombre}")
    print(f"Accuracy: {res['accuracy']:.4f}")
    print(f"Recall clase 1: {res['recall_1']:.4f}")
    print(f"F1-score clase 1: {res['f1_1']:.4f}")
    print(f"Tiempo: {res['tiempo']:.2f} s")



Resumen tuning con 25% de datos:

Modelo: SVM_1_{'kernel': 'rbf', 'C': 0.5, 'gamma': 'scale'}
Accuracy: 0.8242
Recall clase 1: 0.6831
F1-score clase 1: 0.7269
Tiempo: 183.77 s

Modelo: SVM_2_{'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'}
Accuracy: 0.8269
Recall clase 1: 0.6917
F1-score clase 1: 0.7325
Tiempo: 181.99 s

Modelo: SVM_3_{'kernel': 'rbf', 'C': 1.0, 'gamma': 0.1}
Accuracy: 0.8190
Recall clase 1: 0.6663
F1-score clase 1: 0.7161
Tiempo: 251.84 s

Modelo: SVM_4_{'kernel': 'poly', 'C': 1.0, 'gamma': 'scale'}
Accuracy: 0.7999
Recall clase 1: 0.5864
F1-score clase 1: 0.6676
Tiempo: 116.54 s

Modelo: SVM_5_{'kernel': 'linear', 'C': 1.0}
Accuracy: 0.7494
Recall clase 1: 0.5423
F1-score clase 1: 0.5972
Tiempo: 229.17 s


Entrenar el modelo final con el 100% de X_train_scaled

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import time

print("\n⏳ Entrenando el modelo final con todo el conjunto de entrenamiento...")

inicio = time.time()

final_svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
final_svm.fit(X_train_scaled, y_train)

fin = time.time()


⏳ Entrenando el modelo final con todo el conjunto de entrenamiento...


[codecarbon INFO @ 17:19:43] Energy consumed for RAM : 0.000813 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:19:43] Energy consumed for all CPUs : 0.001355 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:19:43] 0.002168 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:19:58] Energy consumed for RAM : 0.000825 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:19:58] Energy consumed for all CPUs : 0.001376 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:19:58] 0.002201 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:20:13] Energy consumed for RAM : 0.000838 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:20:13] Energy consumed for all CPUs : 0.001396 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 17:20:13] 0.002234 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:20:28] Energy consumed for RAM : 0.000850 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 17:20:28] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 5.0 W
[codecarbon

In [12]:
# Predicción en test
y_pred_final = final_svm.predict(X_test_scaled)

# Métricas
acc_final = accuracy_score(y_test, y_pred_final)
report_final = classification_report(y_test, y_pred_final, output_dict=True)
conf_final = confusion_matrix(y_test, y_pred_final)

# Mostrar resultados
print("\n Modelo final entrenado en {:.2f} segundos".format(fin - inicio))
print("\n Classification Report (clase 1):")
print("Accuracy:", round(acc_final, 4))
print("Recall clase 1:", round(report_final['1']['recall'], 4))
print("F1-score clase 1:", round(report_final['1']['f1-score'], 4))
print("\n Matriz de Confusión:")
print(conf_final)


[codecarbon INFO @ 18:54:19] Energy consumed for RAM : 0.001763 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 18:54:19] Energy consumed for all CPUs : 0.002939 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 18:54:19] 0.004702 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:54:34] Energy consumed for RAM : 0.001776 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 18:54:34] Energy consumed for all CPUs : 0.002960 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 18:54:34] 0.004735 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:16:09] Energy consumed for RAM : 0.001788 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:16:09] Energy consumed for all CPUs : 0.002981 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:16:09] 0.004769 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:16:24] Energy consumed for RAM : 0.001801 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:16:24] Energy consumed for all CPUs : 0.003001 kWh. Total CPU Power : 5.0 W
[codecarbon


 Modelo final entrenado en 5679.16 segundos

 Classification Report (clase 1):
Accuracy: 0.8338
Recall clase 1: 0.7036
F1-score clase 1: 0.7437

 Matriz de Confusión:
[[29635  3232]
 [ 5078 12055]]


In [14]:
from sklearn.metrics import roc_auc_score

# Probabilidades para clase 1
y_proba_final = final_svm.decision_function(X_test_scaled)

# Calcular AUC-ROC
auc_final = roc_auc_score(y_test, y_proba_final)
print("\n AUC-ROC:", round(auc_final, 4))



 AUC-ROC: 0.8954


In [13]:
emissions = tracker.stop()

[codecarbon INFO @ 19:22:55] Energy consumed for RAM : 0.002126 kWh. RAM Power : 3.0 W
[codecarbon INFO @ 19:22:55] Energy consumed for all CPUs : 0.003545 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:22:55] 0.005671 kWh of electricity used since the beginning.
